In [1]:
import tensorflow.compat.v1 as tf
import os
import cPickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print os.listdir(CIFAR_DIR)

['data_batch_1', 'readme.html', 'batches.meta', 'data_batch_2', 'data_batch_5', 'test_batch', 'data_batch_4', 'data_batch_3']


In [2]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = cPickle.load(f)
        return data['data'], data['labels']

# refer to tensoflow.Dataset
class CifarData:
    # shuffle: rearrange data, no dependency bw data
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            
            # collect all data of all labels
            all_data.append(data)
            all_labels.append(labels)
            
                    
        # merge np vectors (items) vertically into a matrix
        self._data = np.vstack(all_data)
        
        # normalize
        self._data = self._data / 127.5 - 1
        
        # merge labels horizonally
        self._labels = np.hstack(all_labels)
        
        print self._data.shape
        print self._labels.shape
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        # current image during the loop
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # random permutate array items
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self. _labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print batch_data
# print batch_labels

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [3]:
tf.disable_v2_behavior() 

x = tf.placeholder(tf.float32, [None, 3072])
# [None]
y = tf.placeholder(tf.int64, [None])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32 * 32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# conv_1: neuron graph = feature_map = output image
conv_1 = tf.layers.conv2d(x_image,
                          32, # output channel number
                          (3, 3), # kernel size
                          # same: output image size same;valid: no padding
                          padding = 'same', 
                          activation = tf.nn.relu,
                          name = 'conv1')
# 16 * 16
pooling_1 = tf.layers.max_pooling2d(conv_1,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool1')

conv_2 = tf.layers.conv2d(pooling_1,
                          32, # output channel number
                          (3, 3), # kernel size
                          # same: output image size same;valid: no padding
                          padding = 'same', 
                          activation = tf.nn.relu,
                          name = 'conv2')
# 8 * 8
pooling_2 = tf.layers.max_pooling2d(conv_2,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool2')

conv_3 = tf.layers.conv2d(pooling_2,
                          32, # output channel number
                          (3, 3), # kernel size
                          # same: output image size same;valid: no padding
                          padding = 'same', 
                          activation = tf.nn.relu,
                          name = 'conv3')
# 4 * 4
pooling_3 = tf.layers.max_pooling2d(conv_3,
                                   (2, 2), # kernel size
                                   (2, 2), # stride
                                   name = 'pool3')

# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling_3)

# fully connected
y_ = tf.layers.dense(flatten, 10)

# y_ -> softmax
# y -> one_hot
# loss = y * log y_
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)

# indices
predict = tf.argmax(y_, 1)

# [1, 0, 1, 1, 0, 0]
correct_prediction = tf.equal(predict, y)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [4]:
init = tf.global_variables_initializer()
batch_size = 10
train_steps = 10000
test_steps = 100

# train_op: train model or only infer
# feed_dict: data
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], feed_dict={
                x: batch_data, 
                y: batch_labels})
        if (i+1) % 500 == 0:
            print '[Train] Step: %d, loss: %4.5f, acc: %4.5f' \
                % (i+1, loss_val, acc_val)
        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels = \
                test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy], 
                    feed_dict = {x: test_batch_data, 
                                 y: test_batch_labels})
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print '[Test] Step: %d, acc: %4.5f' % (i+1, test_acc)

[Train] Step: 500, loss: 3.08117, acc: 0.00000
[Train] Step: 1000, loss: 1.77355, acc: 0.30000
[Train] Step: 1500, loss: 1.82143, acc: 0.30000
[Train] Step: 2000, loss: 1.06343, acc: 0.80000
[Train] Step: 2500, loss: 1.44953, acc: 0.40000
[Train] Step: 3000, loss: 1.48539, acc: 0.50000
[Train] Step: 3500, loss: 1.32648, acc: 0.50000
[Train] Step: 4000, loss: 1.03472, acc: 0.70000
[Train] Step: 4500, loss: 1.30668, acc: 0.50000
[Train] Step: 5000, loss: 0.71589, acc: 0.80000
(10000, 3072)
(10000,)
[Test] Step: 5000, acc: 0.62600
[Train] Step: 5500, loss: 1.17395, acc: 0.70000
[Train] Step: 6000, loss: 1.27567, acc: 0.70000
[Train] Step: 6500, loss: 1.16907, acc: 0.70000
[Train] Step: 7000, loss: 1.00991, acc: 0.70000
[Train] Step: 7500, loss: 0.47160, acc: 0.90000
[Train] Step: 8000, loss: 1.20391, acc: 0.30000
[Train] Step: 8500, loss: 1.38505, acc: 0.80000
[Train] Step: 9000, loss: 0.62507, acc: 0.70000
[Train] Step: 9500, loss: 1.00674, acc: 0.60000
[Train] Step: 10000, loss: 0.36620